In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
df="/kaggle/input/fast-food-classification-dataset/Fast Food Classification V2/Train"

In [3]:
datagen=ImageDataGenerator(
    rotation_range=42,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    rescale=1.0/255,
    validation_split=0.2
    
)
train_data=datagen.flow_from_directory(
    df,
    target_size=(256,256),
    batch_size=32,
    subset="training"
)
val_data=datagen.flow_from_directory(
    df,
    target_size=(256,256),
    batch_size=32,
    subset="validation"
)

Found 12000 images belonging to 10 classes.
Found 3000 images belonging to 10 classes.


# ##  CNN# 

In [4]:
from keras.layers import Layer,Dense,Flatten,BatchNormalization,Conv2D,MaxPooling2D,Input
from keras.models import Model,Sequential
from keras.initializers import HeNormal,GlorotUniform
from keras.regularizers import  l2
from keras.optimizers import Adam,RMSprop
from keras.callbacks import EarlyStopping

In [5]:
early_stopping=EarlyStopping(
    monitor="val_loss",
    verbose=1,
    patience=5
)

In [7]:
inputs=Input(shape=(256,256,3))
c=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
c=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
c=MaxPooling2D(pool_size=(2,2),strides=(2,2))(c)
c=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(c)
c=Conv2D(256,(3,3),activation="relu",padding="same",kernel_initializer=GlorotUniform(),kernel_regularizer=l2(0.01))(c)
c=MaxPooling2D(pool_size=(2,2),strides=(2,2))(c)

c=Conv2D(556,(3,3),activation="relu",padding="same",kernel_initializer=GlorotUniform(),kernel_regularizer=l2(0.01))(c)
c=Conv2D(1056,(3,3),activation="relu",padding="same",kernel_initializer=GlorotUniform(),kernel_regularizer=l2(0.01))(c)
c=MaxPooling2D(pool_size=(2,2),strides=(2,2))(c)

c=Conv2D(2056,(3,3),activation="relu",padding="same",kernel_initializer=GlorotUniform(),kernel_regularizer=l2(0.01))(c)
c=Flatten()(c)
c=Dense(32,activation="relu")(c)
outputs=Dense(10,activation="softmax")(c)
model=Model(inputs,outputs)
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()
model.fit(train_data,epochs=15,validation_data=val_data)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 64, 64, 556)    │     1,281,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 64, 64, 1056)   │     5,285,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 32, 32, 1056)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 32, 32, 2056)   │    19,542,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2105344)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │    67,371,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,868,926 (358.08 MB)

 Trainable params: 93,868,926 (358.08 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0937 - loss: 30.2618

2024-12-09 13:05:32.447930: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng20{k2=2,k4=1,k5=0,k6=0,k7=0,k19=0} for conv (f32[24,1056,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[24,556,64,64]{3,2,1,0}, f32[1056,556,3,3]{3,2,1,0}, f32[1056]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-12-09 13:05:32.554969: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.10714986s
Trying algorithm eng20{k2=2,k4=1,k5=0,k6=0,k7=0,k19=0} for conv (f32[24,1056,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[24,556,64,64]{3,2,1,0}, f32[1056,556,3,3]{3,2,1,0}, f32[1056]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$co

375/375 ━━━━━━━━━━━━━━━━━━━━ 504s 1s/step - accuracy: 0.0937 - loss: 30.2171 - val_accuracy: 0.1000 - val_loss: 7.5281
Epoch 2/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 450s 1s/step - accuracy: 0.0982 - loss: 7.0123 - val_accuracy: 0.1000 - val_loss: 5.8380
Epoch 3/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 450s 1s/step - accuracy: 0.1009 - loss: 5.5751 - val_accuracy: 0.1000 - val_loss: 4.9152
Epoch 4/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 449s 1s/step - accuracy: 0.0939 - loss: 4.7503 - val_accuracy: 0.1000 - val_loss: 4.3221
Epoch 5/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 446s 1s/step - accuracy: 0.1011 - loss: 4.2102 - val_accuracy: 0.1000 - val_loss: 3.9147
Epoch 6/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 445s 1s/step - accuracy: 0.0900 - loss: 3.8352 - val_accuracy: 0.1000 - val_loss: 3.6213
Epoch 7/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 446s 1s/step - accuracy: 0.0993 - loss: 3.5619 - val_accuracy: 0.1000 - val_loss: 3.3994
Epoch 8/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 446s 1s/step - accuracy: 0.0924 - loss: 3.3531 - val_accuracy: 0.10